# STO-LTO model creation

authors: [Toma Susi]
date: 2024/09/02

In [108]:
import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np

from ase.build import surface

srtio3 = ase.io.read("data/SrTiO3.cif")

srtio3_110 = surface(srtio3, indices=(1, 1, 0), layers=2, periodic=True)
srtio3_110.wrap()

repeated_srtio3 = srtio3_110.copy()
repeated_srtio3 *= (3, 4, 10)

In [103]:
%matplotlib ipympl
from IPython.display import display
import ipywidgets

# widget figure generation
with plt.ioff():
    dpi = 72
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(675/dpi, 300/dpi), dpi=dpi)

supercell = srtio3_110.copy()
supercell.center(2, axis=2)

top = abtem.show_atoms(supercell, ax=ax1, show_periodic=True)
beam = abtem.show_atoms(supercell, ax=ax2, plane='xz', show_periodic=True, legend=True)
ax1.set_title("Top view")
ax2.set_title("Beam view")

fig.tight_layout()

fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = True
fig.canvas.layout.width = '675px'
fig.canvas.layout.height = '330px'
fig.canvas.toolbar_position = 'bottom'

In [104]:
def update_atoms(x, y, z):
    ax1.cla()
    ax2.cla()
    supercell = srtio3_110*(x,y,z)
    supercell.center(2, axis=2)
    top = abtem.show_atoms(supercell, ax=ax1, show_periodic=True)
    beam = abtem.show_atoms(supercell, ax=ax2, plane='xz', show_periodic=True, legend=True)
    fig.canvas.draw_idle()
    return None

In [105]:
style = {
    'description_width': 'initial',
}

sliderx = ipywidgets.IntSlider(
    orientation='horizontal',
    description='x repetitions: ',
    value=1,
    min=1,
    max=10,
    style = style,
)

slidery = ipywidgets.IntSlider(
    orientation='horizontal',
    description='y repetitions: ',
    value=1,
    min=1,
    max=10,
    style = style,
)

sliderz = ipywidgets.IntSlider(
    orientation='horizontal',
    description='z repetitions: ',
    value=1,
    min=1,
    max=10,
    style = style,
)

ipywidgets.interactive_output(
    update_atoms, 
    {
        'x':sliderx,
        'y':slidery,
        'z':sliderz,
    },
)
None

In [106]:
#| label: app:sto_supercell

widget = ipywidgets.VBox(
    [
        fig.canvas,
            ipywidgets.VBox([
                sliderx,
                slidery,
                sliderz,
            ]),
    ],
)

display(widget);

In [107]:
sto_lto = repeated_srtio3.copy()

mask = sto_lto.symbols == "Sr"

mask = mask * (sto_lto.positions[:, 1] < 7.5)

sto_lto.numbers[mask] = 57

ase.io.write("data/sto_lto.cif", sto_lto)